In [140]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [141]:
# Loading data
data_no_onehot = pd.read_csv('../data/all.csv')
data_onehot = pd.read_csv('../data/all_with_onehot.csv')

In [142]:
data_no_onehot.head()

,Date,Weekday,MealType,Paytm+Cash,Coupons,SemType,Holiday,CouponsMand
0,2022-09-01,3,BreakFast,186.0,117.0,Acad,0,0.0
1,2022-09-01,3,Lunch,293.0,217.0,Acad,0,0.0
2,2022-09-01,3,EveningSnacks,37.0,139.0,Acad,0,0.0
3,2022-09-01,3,Dinner,113.0,220.0,Acad,0,0.0
4,2022-09-02,4,BreakFast,100.0,236.0,Acad,0,0.0


In [143]:
# Preprocessing the dataset
def preprocess(data, to_onehot=True, to_drop_vacation=False):
    if not to_onehot:
        data = data.dropna()
        
    # Converting the date column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Adding day, dayofweek, month and year columns
    data['Day'] = pd.DatetimeIndex(data['Date']).day
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    data['Year'] = pd.DatetimeIndex(data['Date']).year

    # Scaling the data
    # to_normalize = ['Day']
    # scaler = MinMaxScaler()
    # data[to_normalize] = scaler.fit_transform(data[to_normalize])

    # to_normalize = ['Holiday']
    # scaler = StandardScaler()
    # data[to_normalize] = scaler.fit_transform(data[to_normalize])

    # Encoding the categorical data
    if to_onehot:
        categorical_features = ['Weekday', 'Month', 'Year', 'MealType', 'SemType']
        data = pd.get_dummies(data, columns=categorical_features)
    else:
        # Using label encoding
        categorical_features = ['MealType', 'SemType']
        for feature in categorical_features:
            data[feature] = data[feature].astype('category')
            data[feature] = data[feature].cat.codes

    # Dropping vacations (if required)
    if to_drop_vacation:
        data = data[data['Semtype_Vacation'] == 0]

    # Splitting into X and y
    X = data.drop(columns=['Paytm+Cash', 'Coupons'])
    y_paytm = data['Paytm+Cash']
    y_coupons = data['Coupons']
    y_total = data['Paytm+Cash'] + data['Coupons']

    return X, y_paytm, y_coupons, y_total

In [144]:
# Splitting the data into train and test sets (with vacations)
X_no_onehot, y_paytm_no_onehot, y_coupons_no_onehot, y_total_no_onehot = preprocess(data_no_onehot, to_onehot=False)
X_onehot, y_paytm_onehot, y_coupons_onehot, y_total_onehot = preprocess(data_onehot)

# cutoff at 2023-08-31 for no onehot
X_train_no_onehot = X_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
X_test_no_onehot = X_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_paytm_train_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_paytm_test_no_onehot = y_paytm_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_coupons_train_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_coupons_test_no_onehot = y_coupons_no_onehot[X_no_onehot['Date'] > '2023-08-31']
y_total_train_no_onehot = y_total_no_onehot[X_no_onehot['Date'] <= '2023-08-31']
y_total_test_no_onehot = y_total_no_onehot[X_no_onehot['Date'] > '2023-08-31']

# cutoff at 2023-08-31 for onehot
X_train_onehot = X_onehot[X_onehot['Date'] <= '2023-08-31']
X_test_onehot = X_onehot[X_onehot['Date'] > '2023-08-31']
y_paytm_train_onehot = y_paytm_onehot[X_onehot['Date'] <= '2023-08-31']
y_paytm_test_onehot = y_paytm_onehot[X_onehot['Date'] > '2023-08-31']
y_coupons_train_onehot = y_coupons_onehot[X_onehot['Date'] <= '2023-08-31']
y_coupons_test_onehot = y_coupons_onehot[X_onehot['Date'] > '2023-08-31']
y_total_train_onehot = y_total_onehot[X_onehot['Date'] <= '2023-08-31']
y_total_test_onehot = y_total_onehot[X_onehot['Date'] > '2023-08-31']

# Drop the date column
X_train_no_onehot = X_train_no_onehot.drop(columns=['Date'])
X_test_no_onehot = X_test_no_onehot.drop(columns=['Date'])
X_train_onehot = X_train_onehot.drop(columns=['Date'])
X_test_onehot = X_test_onehot.drop(columns=['Date'])

C:\Users\rajad\AppData\Local\Temp\ipykernel_18472\4106291311.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Date'] = pd.to_datetime(data['Date'])
C:\Users\rajad\AppData\Local\Temp\ipykernel_18472\4106291311.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Day'] = pd.DatetimeIndex(data['Date']).day
C:\Users\rajad\AppData\Local\Temp\ipykernel_18472\4106291311.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [145]:
# Defining parameters for grid search
parameters = {
    'n_estimators': [100, 250, 500, 750, 1000],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['sqrt', 'log2', 25, 50, 100, None]
}

In [146]:
def grid_search(X_train, y_train, X_test, y_test, parameters):
   # Grid search for paytm+cash using for loops
    best_parameters = {}

    for n_estimators in parameters['n_estimators']:
        for max_depth in parameters['max_depth']:
            for min_samples_split in parameters['min_samples_split']:
                for min_samples_leaf in parameters['min_samples_leaf']:
                    for max_features in parameters['max_features']:
                        rfr = RandomForestRegressor(n_estimators=n_estimators, 
                                                    max_depth=max_depth, 
                                                    min_samples_split=min_samples_split, 
                                                    min_samples_leaf=min_samples_leaf,
                                                    max_features=max_features, 
                                                    random_state=42)
                        rfr.fit(X_train, y_train)
                        y_pred = rfr.predict(X_test)
                        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                        r2 = r2_score(y_test, y_pred)
                        best_parameters[(rmse, r2)] = (n_estimators, max_depth, min_samples_split, min_samples_leaf)

    # Printing the best parameters with scores
    best_parameters = sorted(best_parameters.items(), key=lambda x: -1 * x[0][1])
    print('Best parameters:')
    print('RMSE:', best_parameters[0][0][0])
    print('R2:', best_parameters[0][0][1])
    print('Parameters:', best_parameters[0][1])

    return best_parameters[0][1]

In [147]:
# Grid search for paytm+cash using onehot
best_parameters_paytm_onehot = grid_search(X_train_onehot, y_paytm_train_onehot, X_test_onehot, y_paytm_test_onehot, parameters)

KeyboardInterrupt: 

In [ ]:
# Grid search for paytm_cash without using onehot
best_parameters_paytm_no_onehot = grid_search(X_train_no_onehot, y_paytm_train_no_onehot, X_test_no_onehot, y_paytm_test_no_onehot, parameters)

In [ ]:
# Grid search for coupons using onehot
best_parameters_coupons_onehot = grid_search(X_train_onehot, y_coupons_train_onehot, X_test_onehot, y_coupons_test_onehot, parameters)

In [ ]:
# Grid search for coupons without using onehot
best_parameters_coupons_no_onehot = grid_search(X_train_no_onehot, y_coupons_train_no_onehot, X_test_no_onehot, y_coupons_test_no_onehot, parameters)

In [ ]:
# Grid search for total using onehot
best_parameters_total_onehot = grid_search(X_train_onehot, y_total_train_onehot, X_test_onehot, y_total_test_onehot, parameters)

In [ ]:
# Grid search for total without using onehot
best_parameters_total_no_onehot = grid_search(X_train_no_onehot, y_total_train_no_onehot, X_test_no_onehot, y_total_test_no_onehot, parameters)